# Import dataset to Bigquery

For this test, only one file from Criteo dataset was uploaded (1 day).  
These tests are run locally on a conda environment and on containers. To install the latest version of the NVTabular software, please refer to https://github.com/NVIDIA-Merlin/NVTabular.

In [3]:
# Download 1 file from Criteo website
import os

from nvtabular.utils import download_file

download_criteo = True
BASE_DIR = '~/data'
input_path = os.path.join(BASE_DIR, "crit_orig")
NUMBER_DAYS = 1